## Paper LBO Model Example

Case Study Source: <http://www.streetofwalls.com/finance-training-courses/private-equity-training/paper-lbo-model-example/>


In [30]:
# Install dependencies
%pip install -r requirements.txt

  Using cached pyarrow-13.0.0.tar.gz (1.0 MB)
  Installing build dependencies ... \^C
anceled
ERROR: Operation cancelled by user

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
# import libs
import polars as pl
import numpy_financial as np

#### 1. Entry Assumptions


In [335]:
# In the case of encapsulating data,
# why not use a class?
entry_multiple = 5
# As a monthly statement I guess just assume year 1 revenue/12 =  revenue per month
# rev_y1 = 100 * 1000 * 1000 / 12
rev_y1 = 100
rev_y1_monthly = rev_y1 / 12
ebitda_margin = 0.4
debt_ratio = 0.6
equity_ratio = 0.4

ebitda_y1 = rev_y1 * ebitda_margin
price_paid = entry_multiple * ebitda_y1

# Debt & Equity Assumptions
debt = debt_ratio * price_paid
equity = equity_ratio * price_paid

# Income Statement Assumptions
rev_growth = 1.1
int_rate = 0.10
tax_rate = 0.4
capex_rate = 0.15
# working_capital = 5 * 1000 * 1000 / 12
# d_and_a = 20 * 1000 * 1000 / 12
working_capital = 5 / 12
d_and_a = 20 / 12

# Project Period
total_months = 12 * 6
months = [month for month in range(1, total_months + 1)]

#### 2. Income Statement


##### Build Income Statement Class:


In [336]:
# Create the DataFrame for the income statement.
# Instead using a spreadsheet-like left-to-right approch,
# treating it as sans-index dataframe would be more benefitial,
# which means a top-to-bottom vertical structure.


class IncomeStatement:
    def __init__(
        self,
        year1_rev_monthly: float,
        rev_growth: float,
        ebitda_margin: float,
        int_rate: float,
        tax_rate: float,
        da: float,
        debt: float,
        total_months: int,
    ) -> None:
        self.year1_rev_monthly = year1_rev_monthly
        self.rev_growth = rev_growth
        self.ebitda_margin = ebitda_margin
        self.int_rate = int_rate
        self.tax_rate = tax_rate
        self.da = da
        self.debt = debt
        self.total_months = total_months

    def months(self) -> pl.DataFrame:
        return pl.DataFrame(
            data={
                "Month": [f"Month {month}" for month in range(1, self.total_months + 1)]
            }
        )

    def revenue(self) -> pl.DataFrame:
        revenue = []
        revenue_dict = {"Revenue": revenue}
        month = 0
        while month < self.total_months:
            if len(revenue) == 0:
                revenue.append(self.year1_rev_monthly)
                month += 1
            elif len(revenue) % 12 == 0:
                revenue.append(revenue[-1] * 1.1)
                month += 1
            else:
                revenue.append(revenue[-1])
                month += 1

        revenue = pl.DataFrame(data=revenue_dict)
        return self.months().with_columns(revenue)

    def ebitda(self) -> pl.DataFrame:
        return self.revenue().with_columns(
            (pl.col("Revenue") * self.ebitda_margin).alias("EBITDA")
        )

    # Assuming amortization is constant for now?
    def ebit(self) -> pl.DataFrame:
        statement = self.ebitda().with_columns(
            pl.lit(self.da).alias("Depreciation & Amortization")
        )
        return statement.with_columns(
            (pl.col("EBITDA") - pl.col("Depreciation & Amortization")).alias("EBIT")
        )

    def int_exp(self) -> pl.DataFrame:
        int_exp = self.debt * (((1 + self.int_rate) ** (1 / 12)) - 1)
        return self.ebit().with_columns(pl.lit(int_exp).alias("Less: Interest Expense"))

    def ebt(self) -> pl.DataFrame:
        return self.int_exp().with_columns(
            (pl.col("EBIT") - pl.col("Less: Interest Expense")).alias("EBT")
        )

    def tax(self) -> pl.DataFrame:
        return self.ebt().with_columns(
            (pl.col("EBT") * self.tax_rate).alias("Less: Tax Payable")
        )

    def net_income(self) -> pl.DataFrame:
        return self.tax().with_columns(
            (pl.col("EBT") - pl.col("Less: Tax Payable")).alias("Net Income")
        )

In [337]:
# Creating an Income Statement instance
income_statement = IncomeStatement(
    rev_y1_monthly,
    rev_growth,
    ebitda_margin,
    int_rate,
    tax_rate,
    d_and_a,
    debt,
    total_months,
)

lbo_is = (
    income_statement.net_income().with_columns(pl.exclude("Month").round(2)).head(25)
)

# If you wish to go for the usual spreadsheet-like approach:
# lbo = lbo.with_columns(pl.exclude("Month").round(2)).transpose(include_header=True, header_name="Month", column_names="Month")
lbo_is

Month,Revenue,EBITDA,Depreciation & Amortization,EBIT,Less: Interest Expense,EBT,Less: Tax Payable,Net Income
str,f64,f64,f64,f64,f64,f64,f64,f64
"""Month 1""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 2""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 3""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 4""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 5""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 6""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 7""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 8""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43
"""Month 9""",8.33,3.33,1.67,1.67,0.96,0.71,0.28,0.43


### 3. Calculate Cumulative Levered Free Cash Flow


#### Again, build a class for this:


In [338]:
class FCF:
    def __init__(
        self,
        capex_rate: float,
        working_capital: float,
        da: float,
        income_statement: IncomeStatement,
    ) -> None:
        self.income_statement = income_statement
        self.capex_rate = capex_rate
        self.working_capital = working_capital
        self.da = da

    def capex(self) -> pl.DataFrame:
        statement = self.income_statement.net_income()
        return statement.with_columns(
            (pl.col("Revenue") * self.capex_rate).alias("Less: Capex")
        )

    def work_cap(self) -> pl.DataFrame:
        statement = self.capex()
        return statement.with_columns(
            (pl.lit(self.working_capital).alias("Less: Working Capital"))
        )

    def da_addback(self) -> pl.DataFrame:
        statement = self.work_cap()
        return statement.with_columns(
            (pl.lit(self.da).alias("Add: Depreciation & Amortization"))
        )

    def fcf(self) -> pl.DataFrame:
        statement = self.da_addback()
        return statement.with_columns(
            (
                pl.col("Net Income")
                + pl.col("Add: Depreciation & Amortization")
                - pl.col("Less: Capex")
                - pl.col("Less: Working Capital")
            ).alias("Monthly FCF")
        )

    def fcf_only(self) -> pl.DataFrame:
        statement = self.fcf()
        return statement.select(
            pl.col("Month"),
            pl.col("Net Income"),
            pl.col("Less: Capex"),
            pl.col("Less: Working Capital"),
            pl.col("Add: Depreciation & Amortization"),
            pl.col("Monthly FCF"),
        )

    def cum_fcf(self) -> pl.DataFrame:
        statement = self.fcf_only()
        return statement.with_columns(
            pl.col("Monthly FCF").cumsum().alias("Cumulative FCF")
        )

In [343]:
# Creating a free cash flow instance
lbo_fcf = FCF(
    capex_rate,
    working_capital,
    d_and_a,
    income_statement=income_statement,
)

lbo_fcf.cum_fcf().with_columns(pl.exclude("Month").round(2)).head(25)

Month,Net Income,Less: Capex,Less: Working Capital,Add: Depreciation & Amortization,Monthly FCF,Cumulative FCF
str,f64,f64,f64,f64,f64,f64
"""Month 1""",0.43,1.25,0.42,1.67,0.43,0.43
"""Month 2""",0.43,1.25,0.42,1.67,0.43,0.85
"""Month 3""",0.43,1.25,0.42,1.67,0.43,1.28
"""Month 4""",0.43,1.25,0.42,1.67,0.43,1.7
"""Month 5""",0.43,1.25,0.42,1.67,0.43,2.13
"""Month 6""",0.43,1.25,0.42,1.67,0.43,2.56
"""Month 7""",0.43,1.25,0.42,1.67,0.43,2.98
"""Month 8""",0.43,1.25,0.42,1.67,0.43,3.41
"""Month 9""",0.43,1.25,0.42,1.67,0.43,3.83


# WIP TO THIS POINT

### CALCULATE ENDING PURCHASE PRICE (EXIT VALUE) AND RETURNS

Cals steps:

        1. Calculate Total Enterprise Value (TEV) at Exit: Take Forward EBITDA at exit (Year 6 EBITDA) along
           with a 5.0x exit multiple to calculate Exit TEV.
        2. Calculate Net Debt at Exit (also known as Ending Debt):
           Beginning Debt – Debt Pay-down = Ending Debt.
           Beginning Debt – Cumulative FCF = Ending Debt

        3. Calculate ending Equity Value (EV) by subtracting Ending Debt from Exit TEV.
        4. Calculate the Multiple-of-Money (MoM) EV return (Ending EV ÷ Beginning EV).


In [17]:
exit_returns = {
    "exit_EBITDA": lbo_is[6].loc["EBITDA"],
    "exit_multiple": entry_ass["entry_multiple"],
    "beginning_debt": entry_ass["debt_portion"],
    "cumulative_FCF": cumulative_fcf,
}
exit_returns

{'exit_EBITDA': 64.42040000000004,
 'exit_multiple': 5,
 'beginning_debt': 120.0,
 'cumulative_FCF': 33.94590000000002}

In [18]:
exit_returns["exit_TEV"] = exit_returns["exit_EBITDA"] * exit_returns["exit_multiple"]
exit_returns["ending_debt"] = (
    exit_returns["beginning_debt"] - exit_returns["cumulative_FCF"]
)
exit_returns["ending_equity"] = exit_returns["exit_TEV"] - exit_returns["ending_debt"]
exit_returns["MoM"] = exit_returns["ending_equity"] / entry_ass["equity_portion"]
exit_returns

{'exit_EBITDA': 64.42040000000004,
 'exit_multiple': 5,
 'beginning_debt': 120.0,
 'cumulative_FCF': 33.94590000000002,
 'exit_TEV': 322.1020000000002,
 'ending_debt': 86.05409999999998,
 'ending_equity': 236.04790000000023,
 'MoM': 2.950598750000003}

### Estimate IRR based on the MoM multiple.

    The following table is useful for estimating IRR based upon 5-year MoM multiples:
    2.0x MoM over 5 years ~15% IRR
    2.5x MoM over 5 years ~20% IRR
    3.0x MoM over 5 years ~25% IRR
    3.7x MoM over 5 years ~30% IRR

    Therefore, we can assume that the implied IRR for the paper LBO case study is approximately 25%, or slightly below
